
# business, world, environment

In [2]:
import requests
import json
import datetime as dt
import pandas as pd

In [3]:
import pytz
import datetime
# 미국 서부 표준시 (PST) 타임존 설정
pst_timezone = pytz.timezone('US/Pacific')

# 현재 시각을 PST 타임존으로 변환
pst_now = now.astimezone(pst_timezone)

# PST 기준 날짜 문자열 형식으로 변환
pst_now_date = pst_now.strftime("%Y-%m-%d")

print(f"현재 PST 날짜: {pst_now_date}")

NameError: name 'now' is not defined

In [ ]:
now = dt.datetime.now()
now_date = now.strftime("%Y-%m-%d")
now_date

'2024-04-22'

## 크롤링 함수

In [ ]:
# my_api_key = "6de253c2-9d98-4b21-9750-e4e9b63c8d28"
# section = "lifeandstyle"
# date = "2024-04-10"

def guardian_search(my_api_key, section, date):
  url = "https://content.guardianapis.com/" + section
  final_url = url + "?from-date=" + date + "&to-date=" + date +"&show-fields=bodyText&page-size=200&api-key=" + my_api_key
  response = requests.get(final_url)
  news_data_json = response.content.decode("utf-8")

  # results 부분만 추출
  news_data_crawl = json.loads(news_data_json)
  news_data_crawl_results = news_data_crawl["response"]["results"]

  # results 에서 원하는 부분만 리스트로 추출
  result_list = []
  for search_crawl_context in news_data_crawl_results:
    result_dict = {}
    # result_dict['sectionId'] = search_crawl_context["sectionId"]
    result_dict['article_public_Date'] = search_crawl_context["webPublicationDate"]
    result_dict['title'] = search_crawl_context["webTitle"]
    result_dict['content_url'] = search_crawl_context["webUrl"]
    result_dict['content'] = search_crawl_context["fields"]["bodyText"]
    result_dict["category_id"] = 3
    result_list.append(result_dict)

  result_list

  # df = pd.DataFrame(result_list)

  return result_list

In [ ]:
my_api_key = "6de253c2-9d98-4b21-9750-e4e9b63c8d28"
section = "business"
date = "2024-04-09"

news_list = guardian_search(my_api_key, section, date)

In [ ]:
!pip install bert-extractive-summarizer

## BertSum 사용해 요약하여 리턴

In [ ]:
from summarizer import Summarizer
summarizer = Summarizer()

summary_list = []

for item in news_list:
    body_text = item['content']
    summary = summarizer(body_text,min_length=200, max_length=400)
    summary_list.append(summary)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
print(summary_list)

['According to the Financial Conduct Authority (FCA), which tracks the number of households in financial difficulties, 5.8 million people reported that they were struggling to pay a large bill in February 2020. On the one hand, huge portions of the population have blitzed through their savings and are now living month-to-month with no cushion to fall back on: the figures showed that 11% of people have no disposable income each month.', 'Analysts believe that the soaring cost of consumer debt for US households could affect president Biden’s chances for re-election, as for the first time on record, interest payments on credit cards and car loans are as big a financial burden for Americans as their mortgage interest. In 2020 he was finally debt-free after paying off $54,000 since his divorce, he says, but then a $12,000 medical bill came in, two HVAC [air conditioning] units in his house needed replacing and some trees removing.', 'Claims that Boeing knew of safety flaws in the manufactur

## 요약한 거 json 파일에 열 추가

In [ ]:
def add_abstracts(news_list, summary_list):

  for i, item in enumerate(news_list):
    # 요약본 추출
    summary = summary_list[i]

    # abstract 열 추가 및 요약본 저장
    item['abstract'] = summary

# abstract 열 추가
add_abstracts(news_list, summary_list)

# 결과 확인 (result_list에는 각 문서의 요약본이 'abstract' 열에 담겨있음)
print(news_list)

[{'sectionId': 'business', 'webPublicationDate': '2024-04-09T23:01:31Z', 'webTitle': 'Paying bills a struggle for 7.4 million UK consumers, regulator finds  ', 'webUrl': 'https://www.theguardian.com/business/2024/apr/10/paying-bills-a-struggle-for-74-million-uk-consumers-regulator-finds', 'bodyText': 'More than 7.4 million people in the UK struggled to pay a bill or a credit repayment in January, according to a financial regulator. The figure is less than last year but is still significantly higher than before the cost of living crisis began. According to the Financial Conduct Authority (FCA), which tracks the number of households in financial difficulties, 5.8 million people reported that they were struggling to pay a large bill in February 2020. The data, which gives an insight into the UK’s personal finances, showed that by January 2023, after the Russian invasion of Ukraine and the subsequent cost of living crisis, the number of people in financial difficulties almost doubled to 10

In [ ]:
import json


with open("/content/drive/MyDrive/2차 프로젝트/NLP model/data/bert.json", "w") as f:
    json.dump(news_list, f, indent=4)